# TP3 - Views, Updates and Design (By Daniel Jorge Deutsch)

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
purchase:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [4]:
%load_ext sql
%sql sqlite:///wine.db

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [5]:
%%sql 

DROP TABLE IF EXISTS producer;

CREATE TABLE producer AS
SELECT DISTINCT 
    NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE 
    NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [6]:
%%sql

DROP TABLE IF EXISTS wine;

CREATE TABLE wine AS
SELECT DISTINCT 
    NV, CRU, DEGRE, MILL
FROM 
    MASTER1
WHERE 
    NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [7]:
%%sql

DROP TABLE IF EXISTS production;

CREATE TABLE production AS
SELECT DISTINCT
    NP, NV, QTE
FROM 
    MASTER1
WHERE 
    NV IS NOT NULL 
    OR NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [8]:
%%sql 

DROP TABLE IF EXISTS client;

CREATE TABLE client AS
SELECT DISTINCT 
    NB, NOM, PRENOM, TYPE
FROM 
    MASTER2
WHERE 
    NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
%%sql

DROP TABLE IF EXISTS purchase;

CREATE TABLE purchase AS
SELECT DISTINCT 
    NB, NV, DATES, LIEU, QTE 
FROM MASTER2
WHERE 
    NB IS NOT NULL
    AND NV IS NOT NULL
    AND DATES IS NOT NULL
    AND LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [10]:
%%sql 

DROP TABLE IF EXISTS location;

CREATE TABLE location AS
SELECT DISTINCT 
    LIEU, REGION
FROM MASTER2
WHERE 
    LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [11]:
%%sql

DROP VIEW IF EXISTS bons_buveurs;

CREATE VIEW bons_buveurs AS
SELECT *
FROM client
WHERE 
    TYPE = 'gros'
    OR TYPE = 'moyen'
ORDER BY NB;

 * sqlite:///wine.db
Done.
Done.


[]

In [12]:
# Test
%sql SELECT * FROM bons_buveurs;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [13]:
%%sql

DROP VIEW IF EXISTS buveurs_asec;

CREATE VIEW buveurs_asec AS
SELECT *
FROM client AS c
WHERE 
    c.NB IS NOT NULL
    AND NOT EXISTS (
        SELECT p.NV
        FROM purchase AS p
        WHERE 
            c.NB = p.NB 
            AND p.NV IS NOT NULL
    );

 * sqlite:///wine.db
Done.
Done.


[]

In [14]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [15]:
%%sql

DROP VIEW IF EXISTS buveurs_achats;

CREATE VIEW buveurs_achats AS
SELECT *
FROM client AS c
WHERE 
    c.NB IS NOT NULL 
    AND NOT EXISTS (
        SELECT a.NB
        FROM buveurs_asec AS a
        WHERE c.NB = a.NB
    )

 * sqlite:///wine.db
Done.
Done.


[]

In [16]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [17]:
%%sql

DROP VIEW IF EXISTS q83pl;

CREATE VIEW q83pl AS
SELECT 
    p.LIEU AS LIEU, w.CRU AS CRU, SUM(p.QTE) AS QTE_BUE
FROM purchase AS p, wine AS w
WHERE w.NV = p.NV
AND strftime('%Y', p.DATES) = '1983'
GROUP BY 1, 2
ORDER BY LIEU;

 * sqlite:///wine.db
Done.
Done.


[]

In [18]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

Yes we can do that because a view is simply the result of a query. The next cell creates a view with the results sorted by the ascending order of QTE.

In [19]:
%%sql

DROP VIEW IF EXISTS q83pl_acending;

CREATE VIEW q83pl_acending AS
SELECT 
    p.LIEU AS LIEU, w.CRU AS CRU, SUM(p.QTE) AS QTE_BUE
FROM purchase AS p, wine AS w
WHERE w.NV = p.NV
AND strftime('%Y', p.DATES) = '1983'
GROUP BY 1, 2
ORDER BY QTE_BUE;

SELECT * FROM q83pl_acending;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,260


___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [20]:
%%sql

DROP TABLE IF EXISTS RBB;

CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs;

 * sqlite:///wine.db
Done.
Done.


[]

In [21]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [22]:
%%sql

SELECT c.NB, c.NOM, c.PRENOM, c.TYPE, SUM(p.QTE) AS total
FROM client AS c, purchase AS p
WHERE 
    c.NB = p.NB 
    AND c.TYPE <> 'gros'
GROUP BY c.NB
HAVING SUM(p.QTE) > 100;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,total
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Update instances

In [23]:
%%sql

UPDATE client
SET TYPE = 'gros'
WHERE NB IN (2, 5, 9, 44);

 * sqlite:///wine.db
4 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

After updating the table client, we can clearly see that the view "bons_buveurs" also changed its content but the table RBB remains unaltered. This happends because of the main difference between views and tables: views are evaluated each time they are called whereas a table remains untouched unless it is specifically updated. 

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [24]:
%%sql

DROP TABLE IF EXISTS RBA;

CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec;

 * sqlite:///wine.db
Done.
Done.


[]

In [25]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [26]:
%%sql

INSERT INTO RBA (NB, NOM, PRENOM, TYPE)  
VALUES (101, 'Deutsch', 'Daniel', 'gros');

 * sqlite:///wine.db
1 rows affected.


[]

In [27]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain


The RBA table was created based on the view "buveurs_asec", but as soon as it was created, there was no more link between them. Given that, any change made to the table RBA won't be replicated in the "buveurs_asec" view. That is the reason why we can see the client with NB=101 in RBA but not in "buveurs_asec".

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

I'd propose the following schema:

- Users(user_id, name, nickname, email, birth_date, address_id);
- Addresses(address_id, street, city, state, country, postal_code);
- Friendships(user1_id, user2_id);
- Posts(post_id, user_id, date, text, attachment, original_post);

In the above schema, we would have the following keys:

- Users -> user_id;
- Addresses -> address_id;
- Friendships -> (user1_id, user2_id);
- Posts -> post_id;

This would garantee the following relationships:

- A user can only have one address, but two users can have the same address;
- A user can have many Friendships with other users;
- A user can have many posts;

The idea is to save on the attribute original_post either the key of the Post the user is replying to or null (in the case o null the Post would be the "original post");

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

With the following view we would be able to display only the original posts made by the user's friends.

In [ ]:
CREATE VIEW display_posts_view AS
SELECT *
FROM posts, users, friendships
WHERE
    posts.original_post IS NULL
    AND (
        posts.user_id IN (SELECT friendships.user1_id FROM friendships WHERE friendships.user2_id = users.user_id)
        OR posts.user_id IN (SELECT friendships.user2_id FROM friendships WHERE friendships.user1_id = users.user_id)
    )
ORDER BY posts.date DESC;

To select relevant posts to display first, I propose to:

- Add to the friendships table an attribute that mesures "how strong the friendship is" (by, for instance, the number of interactions in each other's posts);
- Create a Tags, UserTags and PostsTags tables where:
    - The Tags table contains all the tags created from IA based on the content of each post;
    - The UserTags table contains a score for each tag (the higher the score the more interested the user is in posts with the tags content);
    - The PostsTags table contains a score for each tag (the higher the score, the more the post is related to the tag content);